<table width=100%>
    <tr>
        <td width = 40% align = "left">
            <h3> MTH786 Machine Learning with Python</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Semester A </h3>
        </td>
    </tr>
    <tr>
        <td width = 40% align = "left">
            <h3> Coursework 7</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Dr Nicola Perra </h3>
        </td>
    </tr>
</table>

We start by loading necessary libraries, including NumPy (used for linear algebra calculations) and MatPlotLib (used for visualisation).

In [ ]:
import numpy as np
from numpy.testing import assert_array_almost_equal, assert_array_equal
import matplotlib.pyplot as plt
%matplotlib inline

### Regression analysis for height-weight data via gradient descent method
In this exercise you will work with a data set you are familiar with: $\mathtt{heigh\_weight.csv}$. By completing this exercise you will learn how to solve linear/polynomial regression problems using the gradient descent method. In the second part we will learn ho to solve the LASSO problem using either smoothing technique or proximal maps.

We start by loading data. **Important:** please check that the file $\mathtt{height\_weight.csv}$ is located in the same folder with your Jupyter notebook.


In [ ]:
heights = np.genfromtxt("height_weight.csv",
                        delimiter=",",
                        skip_header=1,
                        usecols=[0])

weights = np.genfromtxt("height_weight.csv",
                        delimiter=",",
                        skip_header=1,
                        usecols=[1])
indices = np.argsort(heights)
heights = heights[indices].reshape((len(heights), 1))
weights = weights[indices].reshape((len(weights), 1))

plt.scatter(heights, weights, s=1)
plt.xlabel('Height', fontsize=16)
plt.xticks(fontsize=16)
plt.ylabel('Weight', fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout;

#### Data standardisation 
In real-world problems we usually get a raw data in the form of $s$ samples each of which is described by numeric several values corresponding to different characteristics of the object. Such a data could be highly non-uniform. The goal of applying $\textit{standardisation}$ is to make sure different features of objects are on almost on the same scale so that each feature is equally important and make it easier to process by most ML algorithms. The result of standardisation is that the features will be rescaled to ensure the mean and the standard deviation to be $0$ and $1$, respectively. This means that for a data given by $\mathbf{X} = \left(
\left(\mathbf{x}^{(1)}\right)^{\top},\left(\mathbf{x}^{(2)}\right)^{\top},\ldots,\left(\mathbf{x}^{(s)}\right)^{\top}
\right) \in \mathbb{R}^{s\times d}$ we define a new, rescaled data as:
$$
\hat{\mathbf{x}}^{(i)}_k = \frac{\mathbf{x}^{(i)}_k - \left\langle \mathbf{x}_k \right\rangle }{\left(\sigma_{\mathbf{x}}\right)_k},
$$
where $\left\langle \mathbf{x}_k \right\rangle = \frac{1}{s}\sum\limits_{j=1}^s \mathbf{x}^{(j)}_k$, and
$\left(\sigma_\mathbf{x}\right)_k = \sqrt{
	\frac{1}{s}\sum\limits_{j=1}^s \left(\mathbf{x}^{(j)}_k-\left\langle \mathbf{x}_k \right\rangle\right)^2}$
are the mean and standard deviation of data vector $\mathbf{x}$. 

Write two functions 
1. **standardise** to standardise the columns of a multi-dimensional array. The function **standardise**	takes the multi-dimensional array *data_matrix* as its input argument. It subtracts the means from each column and divides by the standard deviations. It returns the *standardised_matrix*, the *row_of_means* and the *row_of_standard_deviations*.
2. **de_standardise** to de-standardise the columns of a multi-dimensional array. The function **de_standardise** reverses the above operation. It takes a *standardised_matrix*, the *row_of_means* and the *row_of_standard_deviations* as its arguments and returns a matrix for which the standardisation process is reversed.



In [ ]:
def standardise(data_matrix):
    row_of_means = np.mean(data_matrix, axis=0)
    standardised_matrix = data_matrix - row_of_means
    row_of_stds = np.std(data_matrix, axis=0)
    return (standardised_matrix / row_of_stds), row_of_means, row_of_stds

Test your function with the following unit tests

In [ ]:
test_data_matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
test_standardise_data_matrix = np.array([[-1.224745, -1.224745, -1.224745],
                                         [0., 0., 0.],
                                         [1.224745, 1.224745, 1.224745]])
test_row_of_means = np.array([4, 5, 6])
test_row_of_stds = np.array(np.sqrt([6, 6, 6]))

test_result_standardise_data_matrix, test_result_row_of_means, test_result_row_of_stds = standardise(
    test_data_matrix)
assert_array_almost_equal(test_result_standardise_data_matrix,
                          test_standardise_data_matrix)
assert_array_almost_equal(test_result_row_of_means, test_row_of_means)
assert_array_almost_equal(test_result_row_of_stds, test_row_of_stds)

In [ ]:
def de_standardise(standardised_matrix, row_of_means, row_of_stds):
    matrix = np.copy(standardised_matrix * row_of_stds)
    return matrix + row_of_means

Test your function with the following unit tests

In [ ]:
test_standardise_data_matrix = np.array([[-1.224745, -1.224745, -1.224745],
                                         [0., 0., 0.],
                                         [1.224745, 1.224745, 1.224745]])
test_row_of_means = np.array([4, 5, 6])
test_row_of_stds = np.array(np.sqrt([6, 6, 6]))

assert_array_almost_equal(
    np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    de_standardise(test_standardise_data_matrix, test_row_of_means,
                   test_row_of_stds))

In [ ]:
# let's standardize the data input and output
data_input, input_row_of_means, input_row_of_stds = standardise(heights)
data_output, output_row_of_means, output_row_of_stds = standardise(weights)

#### Building linear/polynomial regressions
In this part you are given a piece of the code from previous assignments that evaluates coefficients of polynomial regression of fixed degree.

In [ ]:
def polynomial_basis(data_inputs, degree):
    X_matrix = np.ones((len(data_inputs), 1))
    for i in range(degree):
        X_matrix = np.c_[X_matrix, np.power(data_inputs, i + 1)]
    return X_matrix


def polynomial_regression(data_matrix, data_outputs):
    return np.linalg.solve(data_matrix.T @ data_matrix,
                           data_matrix.T @ data_outputs)


def prediction_function(data_matrix, weights):
    return data_matrix @ weights

Now use the above functions to evaluate optimal weights in two cases: $d=1$ (linear regression) and $d=5$ (polynomial regression). Plot corresponding results.

In [ ]:
degree_linear = 1
degree_polynomial = 5

data_matrix_linear = polynomial_basis(data_input, degree_linear)
data_matrix_polynomial = polynomial_basis(data_input, degree_polynomial)

optimal_weights_linear = polynomial_regression(data_matrix_linear, data_output)

optimal_weights_polynomial = polynomial_regression(data_matrix_polynomial,
                                                   data_output)

print("Optimal linear regression coefficients are equal to: {w}. \
    \nOptimal polynomial regression (d = {d}) coefficients are equal to: {p}"                                                                             .\
      format(w = optimal_weights_linear.T, d = degree_polynomial, p = optimal_weights_polynomial.T))

**Remark:** please note that an optimal polynomial of degree $5$ is almost equal to a linear function. We will see later that this is not the case for corresponding LASSO problem.

In [ ]:
predictions_linear = de_standardise(prediction_function(data_matrix_linear, optimal_weights_linear),\
                                    output_row_of_means, output_row_of_stds)
predictions_polynomial = de_standardise(prediction_function(data_matrix_polynomial, optimal_weights_polynomial),\
                                       output_row_of_means, output_row_of_stds)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(35, 15))
fig.suptitle('Linear and polynomial regressions for height weight data',
             fontsize=30)
for ax in fig.get_axes():
    ax.scatter(heights, weights, s=2)
    ax.set(xlabel='Height', ylabel='Weight')
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(30)
    ax.label_outer()
ax1.plot(heights, predictions_linear, linewidth=2, color='r')
ax2.plot(heights, predictions_polynomial, linewidth=2, color='r')
plt.savefig('figure.pdf')
plt.tight_layout;

#### Solving regression problem with the gradient descent
The gradient descent is a method for finding a minimum of a function by running an iterative algorithm. Let $E\left(\mathbf{w}\right): \mathbb{R}^n \to \mathbb{R}$ be a real valued, convex, differentiable function of $n$-dimensional argument $\mathbf{w}=\left(w_1,w_2,\ldots,w_n\right)$. Let $\mathbf{w}^{(0)}$ be an arbitrary $n$-dimensional vector. Then the minimiser of the function $E$ can be approximated by consecutively evaluating
$$
\mathbf{w}^{(k+1)} = \mathbf{w}^{(k)} - \tau \nabla E\left(\mathbf{w}^{(k)}\right), \quad k\geq 0,
$$
where $\tau > 0$ is a step-size parameter.

1. Implement a function $\mathtt{gradient\_descent}$ that performs gradient descent to numerically approximate a minimiser of a convex function. The function should take the following arguments
- *objective* - a lambda-function representing function $E$. This itself should take a NumPy array as its argument and return a real number.
- *gradient* - a lambda-function representing function $\nabla E$. This itself should take a NumPy array as its argument and return a NumPy array representation of the gradient $\nabla E$.
- *initial_ weights* - a NumPy array with initial values $\mathbf{w}^{(0)}$ for the first iterate 
- *step_size* - a step-size parameter $\tau$ for the gradient descent step
- *no_of_iterations* - an integer parameter that controls the number of iterations
- *print_output* - an integer parameter that controls how often you are printing an intermediate result. If say *print_output = 100*, then after every 100th iteration you are asked to print your current iterate and a value of the objective as *Iteration k/m, objective = o.*, where $k$ is a number of current iteration, $m$ is a total number of iterations, and $o$ is a value of the objective evaluated at current iterate.

Implement the function so that it returns a NumPy array of the weights obtained after gradient descent together with a list of objective values for all iterates.

In [ ]:
def gradient_descent(objective, gradient, initial_weights, step_size,no_of_iterations=100, print_output=10):
    
    objective_values = [] # this is a list where we will append the value of the energy function
    weights = np.copy(initial_weights) # this is a copy of the initial values of W, why a copy?!
    objective_values.append(objective(weights))
    for counter in range(no_of_iterations):
        weights -= step_size * gradient(weights)
        objective_values.append(objective(weights))
        if (counter + 1) % print_output == 0:
            print("Iteration {k}/{m}, objective = {o}, weights ={w}.".format(k=counter+1, m=no_of_iterations, o=objective_values[counter],w=weights))
        
    return weights, objective_values

### Example 1

Imagine to have an energy function 

$E(x)=x^2-2x+1$

The function is convex. 
The gradient is 

$\nabla E(x)=2x-2$

Hence we know that the minimizer can be obtained imposing the gradient to zero.  This is equivalent to find the argmin
of the Energy function. Hence

$\nabla E(x)=2x-2=0 \rightarrow x=1$

Now let's see whether gradient descent would find the same solution


In [ ]:
test_objective = lambda x: pow(x, 2) - 2 * x + 1  # Energy function
test_gradient = lambda x: 2 * x - 2 #Gradient
test_initial_weights = np.array([0.0])
test_step_size = 0.4
test_no_of_iterations = 10
test_print_output = 2
gradient_descent(test_objective, test_gradient,test_initial_weights,test_step_size,
                                           test_no_of_iterations, test_print_output)[0]

### Example 2

In one of the courseworks we had

$E(x)=\langle X, AX \rangle + \langle v, X \rangle$

The gradient is 

$\nabla E(x)=(A+A^\top)X+v$

$$\langle \mathbf{a},\mathbf{b} \rangle=\mathbf{a}^\top \mathbf{b}$$

In [ ]:
test_matrix_A = np.array([[3, 1], [2, 4]])
test_vector_v = np.array([5, 6])
test_objective = lambda x: x.T @ (test_matrix_A @ x) + test_vector_v.T @ x
test_gradient = lambda x: (test_matrix_A + test_matrix_A.T) @ x + test_vector_v

test_initial_weights = np.array([0.0, 0.0])
test_step_size = 0.9 / (np.linalg.norm(test_matrix_A.T+test_matrix_A)) # remember this comes from the lectures and the 
# conditions of tau-smoothness
test_no_of_iterations = 100
test_print_output = 10
assert_array_almost_equal(gradient_descent(test_objective, test_gradient, \
                                           test_initial_weights,test_step_size,\
                                           test_no_of_iterations, test_print_output)[0],np.array([-0.564103, -0.538462]))

2. Write two functions $\mathtt{mean\_squared\_error}$ and $\mathtt{mean\_squared\_error\_gradient}$ that implement the mean squared error and its gradient as defined below. Both functions take a two-dimensional NumPy array *data_matrix*, a two-dimensional NumPy array *weights* and a two-dimensional NumPy array *data_outputs* as arguments. The first function should return a real number, while the second one should return a matrix representation of the MSE gradient. The MSE function and its gradient are given by
$$
MSE\left(\mathbf{w}\right) = 
\frac{1}{2s}\left\|\mathbf{\Phi}\left(\mathbf{X}\right)\mathbf{w} - \mathbf{Y}\right\|^2,
\quad
\nabla MSE\left(\mathbf{w}\right) = 
\frac{1}{s}\mathbf{\Phi}^{\top}\left(\mathbf{X}\right)\left(\mathbf{\Phi}\left(\mathbf{X}\right)\mathbf{w} - \mathbf{Y}\right),
$$
where $\mathbf{\Phi}\left(\mathbf{X}\right)$ is a mathematical representation of *data_matrix* and $\mathbf{Y}$ is a mathematical representation of *data_output*



In [ ]:
def mean_squared_error(data_matrix, data_outputs, weights):
    return (np.linalg.norm(data_matrix @ weights -data_outputs))**2 / (2 * len(data_matrix))

In [ ]:
def mean_squared_error_gradient(data_matrix, data_outputs, weights):
    return data_matrix.T @ (data_matrix @ weights -data_outputs) / len(data_matrix)

Test your functions with the following unit tests.

In [ ]:
test_data_matrix = np.array([[1, 0.98], [1, 1.02]])
test_data_outputs = np.array([[-0.1], [0.3]])
test_weights = np.array([[-9.9], [10]])
assert_array_almost_equal(
    mean_squared_error(test_data_matrix, test_data_outputs, test_weights), 0)

In [ ]:
test_data_matrix = np.array([[1, 1, -1], [1, 2, 2]])
test_data_outputs = np.array([[-1, 2], [1, 3]])
test_weights = np.array([[0, 0], [1, 2], [3, 4]])
assert_array_almost_equal(
    mean_squared_error(test_data_matrix, test_data_outputs, test_weights),
    36.75)

3. Use the function $\mathtt{gradient\_descent}$ from previous question to minimise the mean squared error for linear and polynomial regression problems discussed in the first section (and thus, to compute their solutions). Take the step-size parameter equal to
$$
\tau = 0.9\cdot s\cdot\left\|\mathbf{\Phi}\left(\mathbf{X}\right)\right\|^{-2},
$$
and the number of iterations equal to $100$ for linear regressions, and $50000$ for polynomial one. Start your iterations from a zero vector of corresponding length. Compare your results with the ones you obtained in the first section.

In [ ]:
objective_linear = lambda weights: mean_squared_error(data_matrix_linear,data_output, weights)
gradient_linear = lambda weights: mean_squared_error_gradient(data_matrix_linear, data_output, weights)

initial_weights_linear = np.zeros((degree_linear + 1, 1))
step_size_linear = 0.9 * len(data_matrix_linear) / (np.linalg.norm(data_matrix_linear))**2

gd_optimal_weights_linear = gradient_descent(objective_linear, gradient_linear,
                                             initial_weights_linear,step_size_linear, 100, 10)[0]

In [ ]:
objective_polynomial = lambda weights: mean_squared_error(data_matrix_polynomial, data_output, weights)
gradient_polynomial = lambda weights: mean_squared_error_gradient(data_matrix_polynomial, data_output, weights)
initial_weights_polynomial = np.zeros((degree_polynomial + 1, 1))
step_size_polynomial = 0.9 * len(data_matrix_polynomial) / (np.linalg.norm(data_matrix_polynomial))**2


gd_optimal_weights_polynomial = gradient_descent(objective_polynomial,
                                                 gradient_polynomial,initial_weights_polynomial,
                                                 step_size_polynomial, 50000,5000)[0]

## Solutions we got applying the normal equation directly
Optimal linear regression coefficients are equal to: [8.57473564e-16 9.24756299e-01].     
Optimal polynomial regression (d = 5) coefficients are equal to: [-1.54785290e-03  1.05220213e+00  4.03175447e-03 -7.60510612e-02
  -4.05488657e-04  6.63445950e-03]

Test your results with the following unit tests

In [ ]:
assert_array_almost_equal(gd_optimal_weights_linear, optimal_weights_linear)

In [ ]:
assert_array_almost_equal(gd_optimal_weights_polynomial,
                          optimal_weights_polynomial)

As one can see from above tests, the weights obtained with gradient descent coincide with the ones obtained by solving the normal equation.

### The LASSO problem for height-weight data


1. Write a function $\mathtt{lasso\_cost\_function}$ that implements the LASSO cost function as introduced in the lecture notes. The arguments include *data_matrix*, *weights* and *outputs* similar to $\mathtt{mean\_squared\_error}$, but further a positive scalar *regularisation_parameter* that controls the balance between mean squared error and the one norm.

In [ ]:
def lasso_cost_function(data_matrix, outputs, weights,regularisation_parameter):
    return mean_squared_error(data_matrix, outputs,weights) + regularisation_parameter * np.sum(np.abs(weights))

Apply your function to the data below

In [ ]:
test_samples, test_dimensions, test_output_dimensions, test_regularisation_parameter = np.random.randint(low=2, high=100, size=4)

test_inputs = np.random.rand(test_samples, test_dimensions)

test_data_outputs = np.random.rand(test_samples, test_output_dimensions)
test_data_matrix = np.c_[np.ones((len(test_inputs), 1)), test_inputs]
test_weights = np.random.rand(test_dimensions + 1, test_output_dimensions)

lasso_cost_function(test_data_matrix,test_data_outputs, test_weights, test_regularisation_parameter)

We now implement two methods of solving the LASSO problem. Namely,
- Smoothing of one-norm
- Proximal gradient descent

#### Smoothing of one-norm


1. Write a function $\mathtt{huber\_loss}$ that evaluates a value of the Huber loss function for a vector. Your function should take a NumPy array *argument* and a scalar *smoothing_parameter* as its arguments. This function is defined as
$$
H_{\tau}\left(\mathbf{w}\right) = 
\sum\limits_{j=0}^{d} \left|w_j\right|_{\tau},\quad
\mbox{with} \quad 
\left|x\right|_{\tau} = 
\begin{cases}
\left|x\right| - \frac{\tau}{2}, &\left|x\right|\geq \tau,\\
\frac{x^2}{2\tau}, &\left|x\right|< \tau,
\end{cases}
$$
where $\tau$ is a mathematical representation of *smoothing_parameter*, while $\mathbf{w}$ is mathematical representation of *argument*.


In [ ]:
def huber_loss(argument, smoothing_parameter):
    
    H=0.
    for x in argument: 
        if np.abs(x)>=smoothing_parameter:
            H+=np.abs(x)-smoothing_parameter/2
        else:
            H+=np.power(x,2)/(2*smoothing_parameter)
    
    return H

Test your function with the following unit tests

In [ ]:
assert_array_almost_equal(huber_loss(np.array([1, 2, 3, -4]), 2), 6.25)

2. Implement the gradient of Huber loss function $\mathtt{huber\_loss\_gradient}$. Your function should take a NumPy array *argument* and a scalar *smoothing_parameter* as its arguments and perform the evaluation of the derivative of Huber loss function
$$
\left|x\right|'_{\tau}
=
\begin{cases}
\mathrm{sgn}\left(x\right), &\left|x\right|\geq \tau,\\
\frac{x}{\tau}, &\left|x\right|< \tau,
\end{cases}
$$
to each component of the array.

In [ ]:
def huber_loss_gradient(argument, smoothing_parameter):
    values=[]
    for x in argument:
        if np.abs(x)>=smoothing_parameter:
            values.append(np.sign(x))
        else:
            values.append(x/smoothing_parameter)
    return np.array(values)

Test your function with the following unit test

In [ ]:
assert_array_almost_equal(huber_loss_gradient(np.array([1, 2, 3, -4]), 2),
                          np.array([0.5, 1., 1., -1.]))

3. Use the function $\mathtt{gradient\_descent}$ from previous question to minimise the lasso cost function for linear and polynomial regression problems discussed in the first section (and thus, to compute their solutions). Take the gradient step-size parameter equal to
$$
\tau = 0.45\cdot s\cdot\left\|\mathbf{\Phi}\left(\mathbf{X}\right)\right\|^{-2},
$$
while assign the Huber loss smoothing parameter with $\hat\tau = 2\cdot \alpha\cdot\tau$, where the regularisation parameter $\alpha$ is equal to $\frac{1}{2}$. Take the number of iterations equal to $1000$ for linear regressions, and $50000$ for polynomial one. Start your iterations from a zero vector of corresponding length. Compare your results with the ones you obtained in the first section.

In [ ]:
test_regularisation_parameter = 0.5
test_step_size_linear = 0.45 * len(data_matrix_linear) / ((np.linalg.norm(data_matrix_linear))**2)
test_smoothing_parameter_linear = 2* test_step_size_linear * test_regularisation_parameter

# the cost function is the LASSO expression
test_objective_linear = lambda weights: lasso_cost_function(data_matrix_linear, data_output, weights, test_regularisation_parameter)
# the gradient is the sum of the gradient of MSE plus the gradient of the Huber loss
test_gradient_linear = lambda weights: (mean_squared_error_gradient(data_matrix_linear, data_output, weights)+ 
                                        test_regularisation_parameter * 
                                        huber_loss_gradient(weights, test_smoothing_parameter_linear))

test_initial_weights_linear = np.zeros((degree_linear + 1, 1)) # initital vector


# we then apply gradient descent
hl_optimal_weights_linear = gradient_descent(test_objective_linear,
                                             test_gradient_linear,
                                             test_initial_weights_linear,
                                             test_step_size_linear, 1000,
                                             100)[0]

In [ ]:
test_regularisation_parameter = 0.5
test_step_size_polynomial = 1.9 * len(data_matrix_polynomial) / ((np.linalg.norm(data_matrix_polynomial))**(2))
test_smoothing_parameter_polynomial = 2*test_step_size_polynomial * test_regularisation_parameter
test_objective_polynomial = lambda weights: lasso_cost_function(data_matrix_polynomial, data_output, weights, test_regularisation_parameter)

test_gradient_polynomial = lambda weights: (mean_squared_error_gradient(
data_matrix_polynomial, data_output, weights
) + test_regularisation_parameter * huber_loss_gradient(
weights, test_smoothing_parameter_polynomial))
test_initial_weights_polynomial = np.zeros((degree_polynomial + 1, 1))

hl_optimal_weights_polynomial = gradient_descent(
    test_objective_polynomial, test_gradient_polynomial,
    test_initial_weights_polynomial, test_step_size_polynomial, 50000, 5000)[0]

In [ ]:
print("Optimal linear regression coefficients are equal to: {l}. \
    \nOptimal polynomial regression (d = {d}) coefficients are equal to: {p}"                                                                             .\
      format(l = hl_optimal_weights_linear.T, d = degree_polynomial,p = hl_optimal_weights_polynomial.T))

## Note: 
Remember that LASSO will "sparsify" the solution, hence in case of small number of variables (like here) it might not be the optimal choice

Test your results with the following unit tests

In [ ]:
assert_array_almost_equal(hl_optimal_weights_linear,
                          np.array([[1.372813e-16], [4.247563e-01]]))

In [ ]:
assert_array_almost_equal(
    hl_optimal_weights_polynomial,
    np.array([[-6.045255e-05], [4.988579e-03], [-1.228225e-05],
              [1.92627433e-01], [-3.09616971e-04], [1.77316147e-03]]))

You can now observe that a polynomial function is very different from a linear one. It is rather qubic now. We finish by plotting resulting regression functions.

In [ ]:
hl_predictions_linear = de_standardise(prediction_function(data_matrix_linear, hl_optimal_weights_linear),\
                                    output_row_of_means, output_row_of_stds)
hl_predictions_polynomial = de_standardise(prediction_function(data_matrix_polynomial, hl_optimal_weights_polynomial),\
                                       output_row_of_means, output_row_of_stds)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(35, 15))
fig.suptitle('Linear and polynomial regressions for height weight data',
             fontsize=30)
for ax in fig.get_axes():
    ax.scatter(heights, weights, s=2)
    ax.set(xlabel='Height', ylabel='Weight')
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(30)
    ax.label_outer()
ax1.plot(heights, hl_predictions_linear, linewidth=2, color='r')
ax2.plot(heights, hl_predictions_polynomial, linewidth=2, color='r')
plt.tight_layout;

#### Proximal gradient descent
In this part we extend the concept of gradient descent to proximal gradient descent. Subsequently, we use proximal gradient descent to implement the LASSO problem introduced in the lecture.

1. Based on your function $\mathtt{gradient\_descent}$ from above, write a function $\mathtt{proximal\_gradient \_descent}$ that takes all the arguments of $\mathtt{gradient\_descent}$ but also a function *proximal_map*.

In [ ]:
def proximal_gradient_descent(objective, gradient, proximal_map,
                              initial_weights, step_size=1, no_of_iterations=100, print_output=10):
    objective_values = []
    weights = initial_weights
    objective_values.append(objective(weights))
    
    for counter in range(no_of_iterations):
        weights = proximal_map(weights - step_size * gradient(weights)) # the weights are the iterative output here
        objective_values.append(objective(weights))
        if (counter + 1) % print_output == 0:
            print("Iteration {k}/{m}, objective = {o}.".format(k=counter+1, m=no_of_iterations, o=objective_values[counter]))
    print("Iteration completed after {k}/{m}, objective = {o}.".format(k=counter+ 1, m=no_of_iterations, o=objective_values[counter]))
    return weights, objective_values

Apply your function below

In [ ]:
test_matrix_m = np.array([[3, 1], [2, 4]])
test_vector_v = np.array([5, 6])

test_objective = lambda x: x.T @ (test_matrix_m @ x) + x @ test_vector_v + 2 * np.linalg.norm(x, ord=1)
test_gradient = lambda x: (test_matrix_m + test_matrix_m.T) @ x + test_vector_v
# why is this the gradient? Remember: the proximal map is applied to w-tau*Nabla(L), where E(w)=L(w)+R(w)!

test_initial_weights = np.array([0.0, 0.0])
test_step_size = 0.9 / (np.linalg.norm(test_matrix_m + test_matrix_m.T))
test_no_of_iterations = 1000
test_print_output = 100

test_proximal_map = lambda x: np.sign(x) * np.maximum(0,np.abs(x) - test_step_size)

proximal_gradient_descent(test_objective,test_gradient, 
                                                    test_proximal_map,test_initial_weights,test_step_size,
                                                    test_no_of_iterations,test_print_output)[0]

2. Implement the proximal map for the one-norm, also known as soft thresholding. Write a function $\mathtt{proximal\_map}$ that takes a NumPy array *argument* and a scalar *threshold* as its arguments and performs the soft-thresholding operation to each component of the array. This is defined as
$$
\mathrm{soft}_{\tau}\left(x\right) = 
x - \tau\cdot H'_{\tau}\left(x\right) = 
\begin{cases}
x - \mathrm{sgn}\left(x\right)\cdot\tau, &\left|x\right|\geq \tau,\\
0, &\left|x\right| < \tau.
\end{cases}
$$

In [ ]:
def soft_thresholding(argument, threshold):
    return np.sign(argument) * np.maximum(0, np.abs(argument) - threshold)

3. Use the function $\mathtt{proximal\_gradient\_descent}$ from previous question to minimise the lasso cost function for linear and polynomial regression problems discussed in the above. Take the gradient step-size parameter equal to
$$
\tau = 0.9\cdot s\cdot\left\|\mathbf{\Phi}\left(\mathbf{X}\right)\right\|^{-2},
$$
while assign the proximal map smoothing parameter with $\hat\tau = \alpha\cdot\tau$, where the regularisation parameter $\alpha$ is equal to $\frac{1}{2}$. Take the number of iterations equal to $1000$ for linear regressions, and $50000$ for polynomial one. Start your iterations from a zero vector of corresponding length. Compare your results with the ones you obtained in the first section.

In [ ]:
test_regularisation_parameter = 0.5
test_step_size_linear = 0.9 * len(data_matrix_linear) * ((np.linalg.norm(data_matrix_linear))**(-2))

test_smoothing_parameter_linear = test_step_size_linear *test_regularisation_parameter

test_objective_linear = lambda weights: lasso_cost_function(data_matrix_linear, data_output, weights, test_regularisation_parameter)
test_gradient_linear = lambda weights: mean_squared_error_gradient(data_matrix_linear, data_output, weights)
test_proximal_map_linear = lambda weights: soft_thresholding(weights, test_smoothing_parameter_linear)

test_initial_weights_linear = np.zeros((degree_linear + 1, 1))

pm_optimal_weights_linear = proximal_gradient_descent(
    test_objective_linear, test_gradient_linear, test_proximal_map_linear,
    test_initial_weights_linear, test_step_size_linear, 1000, 100)[0]

Test your result with the following unit test

In [ ]:
assert_array_almost_equal(pm_optimal_weights_linear,
                          np.array([[-0.], [0.424756]]))

In [ ]:
test_regularisation_parameter = 0.5
test_step_size_polynomial = 0.9 * len(data_matrix_polynomial) * ((np.linalg.norm(data_matrix_polynomial))**(-2))
test_smoothing_parameter_polynomial = test_step_size_polynomial *test_regularisation_parameter

test_objective_polynomial = lambda weights: lasso_cost_function(data_matrix_polynomial, data_output, weights, test_regularisation_parameter)
test_gradient_polynomial = lambda weights: mean_squared_error_gradient(data_matrix_polynomial, data_output, weights)
test_proximal_map_polynomial = lambda weights: soft_thresholding(weights, test_smoothing_parameter_polynomial)

test_initial_weights_polynomial = np.zeros((degree_polynomial + 1, 1))

pm_optimal_weights_polynomial = proximal_gradient_descent(
    test_objective_polynomial, test_gradient_polynomial,
    test_proximal_map_polynomial, test_initial_weights_polynomial,
    test_step_size_polynomial, 50000, 5000)[0]

Test your result with the following unit test

In [ ]:
assert_array_almost_equal(pm_optimal_weights_polynomial,
                          np.array([[0], [0], [0], [0.17920935], [0], [0]]))

We would like to emphasize now that the optimal polynomial has a form $p\left(x\right) = 0.17920395\cdot x^3$. This is in complete agreement with the LASSO method: all coefficients except one are zero. We finish by plotting optimal linear and optimal polynomial regression functions. 

In [ ]:
pm_predictions_linear = de_standardise(prediction_function(data_matrix_linear, pm_optimal_weights_linear),\
                                    output_row_of_means, output_row_of_stds)
pm_predictions_polynomial = de_standardise(prediction_function(data_matrix_polynomial, pm_optimal_weights_polynomial),\
                                       output_row_of_means, output_row_of_stds)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(35, 15))
fig.suptitle('Linear and polynomial regressions for height weight data',
             fontsize=30)
for ax in fig.get_axes():
    ax.scatter(heights, weights, s=2)
    ax.set(xlabel='Height', ylabel='Weight')
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(30)
    ax.label_outer()
ax1.plot(heights, pm_predictions_linear, linewidth=2, color='r')
ax2.plot(heights, pm_predictions_polynomial, linewidth=2, color='r')
plt.tight_layout;